# Movie rewievs

In [1]:
import pandas as pd
import numpy as np


In [4]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, Embedding

Using TensorFlow backend.


In [146]:
from keras.layers import SpatialDropout1D

## data exploration

In [66]:
data_neg = pd.read_csv("./movie_reviews/movie_reviews/train/neg_rating.csv")

In [67]:
data_pos = pd.read_csv("./movie_reviews/movie_reviews/train/pos_rating.csv")

Load the GloVe corpus

In [12]:
embeddings_index = {}
f = open('./glove.6B/glove.6B.300d.txt')
for line in f:
    values = line.split(' ')
    word = values[0] 
    coefs = np.asarray(values[1:], dtype='float32') 
    embeddings_index[word] = coefs
f.close()


In [93]:
def data_loader(data, sentiment):
    train = []
    for i  in range(data.shape[0]):
        file = open("./movie_reviews/movie_reviews/train/{0}/{1}.txt".format(sentiment,i), "r")
        train.append(file.readlines())
    return train

In [168]:
d = data_loader(data_neg, "neg")

In [169]:
d_pos = data_loader(data_pos, "pos")

In [178]:
data = d+d_pos

In [35]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/ira/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [171]:
import re
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
def removing_stop_words(data):
    lines_without_stopwords=[] 
    for seq in data: 
        seq = seq[0].lower()
        seq_by_words = re.findall(r'(?:\w+)', seq, flags = re.UNICODE) 
        new_line=[]
        for word in seq_by_words:
            if word not in stop:
                new_line.append(word)
        lines_without_stopwords.append(new_line)

    return lines_without_stopwords

In [179]:
data = removing_stop_words(data)

In [180]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
MAX_NUM_WORDS = 4000
MAX_SEQUENCE_LENGTH = 400
def prep_with_tokenize_pad(texts):
   
    tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts)

    word_index = tokenizer.word_index
    print('Found %s unique tokens.' % len(word_index))

    data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

   
    
    print("data shape: ", data.shape)

    return word_index, data

In [181]:
word_index_data, data = prep_with_tokenize_pad(data)

Found 76906 unique tokens.
data shape:  (25000, 400)


In [77]:
labels = [0 if i < 12500 else 1 for i in range(data_neg.shape[0]+data_pos.shape[0]) ]

In [101]:
sum(labels)

12500

In [57]:
from keras.initializers import Constant
def embed(word_index, data):
    EMBEDDING_DIM = embeddings_index.get('a').shape[0]
    num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
    embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
    for word, i in word_index.items():
        if i > MAX_NUM_WORDS:
            continue
        embedding_vector = embeddings_index.get(word) 
        if embedding_vector is not None:

            embedding_matrix[i] = embedding_vector
    return embedding_matrix, num_words

In [111]:
#as we have fake labels for train I should split it on test and train by my own
from sklearn.model_selection import train_test_split
train, test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [112]:
embedding_matrix_t, num_words_t = embed(word_index_data, train)

## binary classification

In [85]:

model = Sequential()
model.add(Embedding(num_words, 300, input_length=400, weights= [embedding_matrix], trainable=False))

model.add(Dropout(0.2))
model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [115]:
model.fit(train, np.array(y_train), validation_split=0.1, epochs = 6)

Train on 18000 samples, validate on 2000 samples
Epoch 1/6
18000/18000 [==============================] - 65s 4ms/step - loss: 0.2641 - acc: 0.8909 - val_loss: 0.3017 - val_acc: 0.8710
Epoch 2/6
18000/18000 [==============================] - 65s 4ms/step - loss: 0.2168 - acc: 0.9145 - val_loss: 0.2861 - val_acc: 0.8740
Epoch 3/6
18000/18000 [==============================] - 65s 4ms/step - loss: 0.1676 - acc: 0.9348 - val_loss: 0.2979 - val_acc: 0.8730
Epoch 4/6
18000/18000 [==============================] - 65s 4ms/step - loss: 0.1355 - acc: 0.9476 - val_loss: 0.2883 - val_acc: 0.8820
Epoch 5/6
18000/18000 [==============================] - 65s 4ms/step - loss: 0.1123 - acc: 0.9579 - val_loss: 0.3130 - val_acc: 0.8870
Epoch 6/6
18000/18000 [==============================] - 65s 4ms/step - loss: 0.0843 - acc: 0.9667 - val_loss: 0.3938 - val_acc: 0.8705


In [116]:
scores = model.evaluate(train, y_train, verbose=0)
print(scores)

[0.10220872561708093, 0.9625]


In [117]:
scores = model.evaluate(test, y_test, verbose=0)
print(scores)

[0.40757103457450866, 0.8658]


# rating prediction


One hot labels encoding

In [118]:
 from sklearn.preprocessing import OneHotEncoder

In [134]:
data_cocn = pd.concat([data_neg, data_pos])

In [141]:
def convert_to_one_hot():
    dummies = pd.get_dummies(data_cocn['rating'], prefix = None)
    return dummies.values

In [182]:
Y = convert_to_one_hot()
train_m, test_m, y_train_m, y_test_m = train_test_split(data, Y, test_size=0.2, random_state=42)

In [183]:
embedding_matrix_m, num_words_m = embed(word_index_data, train_m)

In [ ]:
model = Sequential()
model.add(Embedding(num_words, 300,  input_length=400, weights= [embedding_matrix_m], trainable=False))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(8, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


batch_size = 64

fit = model.fit(train_m, train_m, epochs=6, batch_size=batch_size,validation_split=0.1)

In [ ]:
scores = model.evaluate(test, y_test, verbose=0)
print(scores)

In [144]:
Y.shape

(25000, 8)